# Color palette

Aswin van Woudenberg (https://www.aswinvanwoudenberg.com | https://github.com/afvanwoudenberg)

## Introduction

This notebook allows you to upload an image and get a custom set of colors generated based on the most dominant colors in the image.

## Import libraries

We start by importing the necessary libraries. The k-means clustering algorithm is used to calculate the mean color of each cluster to produce a palette.

In [3]:
%matplotlib inline
import cv2
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import squarify
import os

from io import BytesIO
from IPython.display import display, clear_output
from sklearn.cluster import KMeans
from PIL import Image

## Image processing

We will output the hex value for the color of each cluster.

In [4]:
def rgb_to_hex(r, g, b):
    return "#{:02x}{:02x}{:02x}".format(r, g, b)

The next function takes care of the bulk of the work. Firstly it removes any transparent pixels and reshapes the image into a list of pixel values. This list of pixels is then fed into the sklearn KMeans algorithm together with the number of clusters/colors. After the pixels are clustered by similarity we draw a treemap. Optionally, the hex values of the clusters are used as the labels in the treemap.

In [5]:
def update_palette(image, n_clusters, output, show_labels=True):
    # construct an array from the bytes and isolate the non-transparent pixels
    x = np.frombuffer(image.value, dtype='uint8')
    img = cv2.imdecode(x, cv2.IMREAD_UNCHANGED)
    r, k, d = img.shape
    if d == 4:
        pixels = img[img[:,:,3] == 255]
        pixels = pixels[:,:3]
    else:
        pixels = img.reshape((r * k, d))
    
    # cluster pixels
    clt = KMeans(n_clusters = n_clusters)
    clt.fit(pixels)
    
    # show our color chart
    with output:
        clear_output(wait=True)
        labels = [rgb_to_hex(int(r), int(g), int(b)) for b, g, r in clt.cluster_centers_]
        _, sizes = np.unique(clt.labels_, return_counts=True)
        if show_labels:
            squarify.plot(sizes=sizes, label=labels, color=labels)
        else:
            squarify.plot(sizes=sizes, color=labels)
        plt.axis('off')
        plt.show()

## A user interface

We create a `FileUpload`, `BoundedIntText`, `Checkbox` and `Image` widget as our GUI. This allows us to select and upload an image, set the number of clusters/colors, and choose to show or hide the hex color values. The `Image` widget shows the file that has been uploaded. Additionally, a `HBox` is created to horizontally outline the widgets. Furthermore, we create an `Output` widget for our treemap to be drawn in.

In [6]:
fileUpload = widgets.FileUpload(
    multiple = False
)
boundedIntText = widgets.BoundedIntText(
    value = 5,
    min = 2,
    max = 15,
    step = 1,
    description = 'Clusters:',
    disabled = True
)
checkbox = widgets.Checkbox(
    value = True,
    description = 'Show hex codes:',
    disabled = True
)
hBox = widgets.HBox([fileUpload, boundedIntText, checkbox])
image = widgets.Image(
    layout=widgets.Layout(display='none', width='500px')
)
output = widgets.Output()

Next, we'll add some interactivity to our widgets. On every statechange, the `update_palette` function above is called to recalculate the clustering of pixels.

In [7]:
def on_checkbox_changed(c):
    update_palette(image, boundedIntText.value, output, c.new)

checkbox.observe(on_checkbox_changed, 'value')

In [8]:
def on_bounded_int_text_changed(c):
    update_palette(image, c.new, output, checkbox.value)
    
boundedIntText.observe(on_bounded_int_text_changed, 'value')

In [9]:
def on_file_upload_change(c):
    # display image
    [uploaded_file] = fileUpload.value
    image.value = fileUpload.value[uploaded_file]['content']
    image.layout.display = 'inline-block'
    boundedIntText.disabled = False
    checkbox.disabled = False
    update_palette(image, boundedIntText.value, output, checkbox.value)
    
fileUpload.observe(on_file_upload_change, 'value')

The next line displays the GUI.

In [10]:
display(hBox, image, output)

Image(value=b'', layout="Layout(display='none', width='500px')")

Output()

Create a color palette by uploading an image and select the number of colors to cluster by. Depending on the number of pixels processing might take a while.